In [43]:
from gym import Env
import numpy as np
import random

In [44]:
class shower(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space=Box(low=np.array([0]),high=np.array([100]))
        self.state=39+random.randint(-3,3)
        self.shower_length=60
    def step(self,action):
        self.state+=action-1
        self.shower_length-=1
        
        if self.state>=37 and self.state<=39:
            reward=1
        else:
            reward=-1
        if self.shower_length<=0:
            done=True
        else:
            done=False
        #temprature noise
        self.state+=random.randint(-1,1)
        info={}
        return self.state, reward, done, info
    def render(self):
        pass
    def reset(self):
        self.state=39+random.randint(-3,3)
        self.shower_length=60
        return self.state

In [45]:
env=shower()

D:\ImageClassification\imageclassification\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [46]:
env.observation_space.sample()

array([46.118923], dtype=float32)

In [47]:
ep=10
for episode in range(1,ep+1):
    state=env.reset()
    done=False
    score=0
    
    while not done:
        env.render()
        action=env.action_space.sample()
        n_state,reward,done,info=env.step(action)
        score+=reward
    print('episode:{}   Score:{}'.format(episode,score))

episode:1   Score:-12
episode:2   Score:-60
episode:3   Score:-18
episode:4   Score:-60
episode:5   Score:-24
episode:6   Score:-60
episode:7   Score:-28
episode:8   Score:-60
episode:9   Score:-46
episode:10   Score:-56


In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [49]:
states=env.observation_space.shape
actions=env.action_space.n

In [50]:
def Model(states,actions):
    model=Sequential()
    model.add(Dense(24,activation='relu',input_shape=states))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [51]:
model=Model(states,actions)

In [52]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 24)                48        
                                                                 
 dense_8 (Dense)             (None, 24)                600       
                                                                 
 dense_9 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [53]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from keras.optimizers import Adam

In [54]:
def Agent(model,actions):
    policy=BoltzmannQPolicy()
    memory=SequentialMemory(limit=50000,window_length=1)
    dqn=DQNAgent(model=model,memory=memory,policy=policy,
                 nb_actions=actions,nb_steps_warmup=10,target_model_update=1e-2)
    return dqn

In [55]:
dqn=Agent(model,actions)
dqn.compile(Adam(lr=1e-3),metrics=['mae'])
dqn.fit(env,nb_steps=50000,visualize=False,verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


D:\ImageClassification\imageclassification\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 4:28:49 - reward: 1.0000

D:\ImageClassification\imageclassification\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 128s 13ms/step - reward: -0.6200
166 episodes - episode_reward: -37.133 [-60.000, 14.000] - loss: 0.898 - mae: 6.198 - mean_q: -8.153

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 126s 13ms/step - reward: -0.5402
167 episodes - episode_reward: -32.575 [-60.000, 18.000] - loss: 1.849 - mae: 10.081 - mean_q: -14.407

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 121s 12ms/step - reward: -0.5418
167 episodes - episode_reward: -32.431 [-60.000, 36.000] - loss: 2.882 - mae: 12.611 - mean_q: -18.189

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 110s 11ms/step - reward: -0.5492
166 episodes - episode_reward: -32.867 [-60.000, 20.000] - loss: 3.018 - mae: 12.974 - mean_q: -18.744

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 118s 12ms/step - reward: -0.5560
done, took 601.756 seconds
